In [30]:
import requests
from bs4 import BeautifulSoup
from retry import retry
import urllib

import pandas as pd
import numpy as np
import copy
import time
import re
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib

In [2]:
# 音を出す
def sound():
    !rundll32 user32.dll,MessageBeep

In [118]:
# 複数ページの情報をまとめて取得
data_samples = []

# スクレイピングするページ数
max_page = 2000
# SUUMOを東京都23区のみ指定して検索して出力した画面のurl(ページ数フォーマットが必要)
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=25&pc=50&page={}'

# リクエストがうまく行かないパターンを回避するためのやり直し
@retry(tries=3, delay=10, backoff=2)
def load_page(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

# 処理時間を測りたい
start = time.time()
times = []

# ページごとの処理
for page in range(1,max_page+1):
    before = time.time()
    # ページ情報
    soup = load_page(url.format(page))
    # 物件情報リストを指定
    mother = soup.find_all(class_='cassetteitem')
    
    # 物件ごとの処理
    for child in mother:

        # 建物情報
        data_home = []
        # カテゴリ
        data_home.append(child.find(class_='ui-pct ui-pct--util1').text)
        # 建物名
        data_home.append(child.find(class_='cassetteitem_content-title').text)
        # 住所
        data_home.append(child.find(class_='cassetteitem_detail-col1').text)
        # 最寄り駅のアクセス
        children = child.find(class_='cassetteitem_detail-col2')
        for id,grandchild in enumerate(children.find_all(class_='cassetteitem_detail-text')):
            data_home.append(grandchild.text)
        # 築年数と階数
        children = child.find(class_='cassetteitem_detail-col3')
        for grandchild in children.find_all('div'):
            data_home.append(grandchild.text)

        # 部屋情報
        rooms = child.find(class_='cassetteitem_other')
        for room in rooms.find_all(class_='js-cassette_link'):
            data_room = []
            
            # 部屋情報が入っている表を探索
            for id_, grandchild in enumerate(room.find_all('td')):
                # 階
                if id_ == 2:
                    data_room.append(grandchild.text.strip())
                # 家賃と管理費
                elif id_ == 3:
                    data_room.append(grandchild.find(class_='cassetteitem_other-emphasis ui-text--bold').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--administration').text)
                # 敷金と礼金
                elif id_ == 4:
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--deposit').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--gratuity').text)
                # 間取りと面積
                elif id_ == 5:
                    data_room.append(grandchild.find(class_='cassetteitem_madori').text)
                    data_room.append(grandchild.find(class_='cassetteitem_menseki').text)
                # url
                elif id_ == 8:
                    get_url = grandchild.find(class_='js-cassette_link_href cassetteitem_other-linktext').get('href')
                    abs_url = urllib.parse.urljoin(url,get_url)
                    data_room.append(abs_url)
            # 物件情報と部屋情報をくっつける
            data_sample = data_home + data_room
            data_samples.append(data_sample)
    
    # 1アクセスごとに1秒休む
    time.sleep(1)
    
    # 進捗確認
    # このページの作業時間を表示
    after = time.time()
    running_time = after - before
    times.append(running_time)
    print(f'{page}ページ目：{running_time}秒')
    # 取得した件数
    print(f'総取得件数：{len(data_samples)}')
    # 作業進捗
    complete_ratio = round(page/max_page*100,3)
    print(f'完了：{complete_ratio}%')
    # 作業の残り時間目安を表示
    running_mean = np.mean(times)
    running_required_time = running_mean * (max_page - page)
    hour = int(running_required_time/3600)
    minute = int((running_required_time%3600)/60)
    second = int(running_required_time%60)
    print(f'残り時間：{hour}時間{minute}分{second}秒\n')

# 処理時間を測りたい
finish = time.time()
running_all = finish - start
print('総経過時間：',running_all)
sound()

# 2000ページ以降はSUUMO側が耐えないらしいよ

1ページ目：7.05992865562439秒
総取得件数：135
完了：0.05%
残り時間：3時間55分12秒

2ページ目：6.722849130630493秒
総取得件数：300
完了：0.1%
残り時間：3時間49分28秒

3ページ目：6.978183031082153秒
総取得件数：524
完了：0.15%
残り時間：3時間50分19秒

4ページ目：6.869519472122192秒
総取得件数：789
完了：0.2%
残り時間：3時間49分47秒

5ページ目：6.590523719787598秒
総取得件数：1087
完了：0.25%
残り時間：3時間47分34秒

6ページ目：6.296193599700928秒
総取得件数：1264
完了：0.3%
残り時間：3時間44分25秒

7ページ目：8.380576610565186秒
総取得件数：1603
完了：0.35%
残り時間：3時間52分1秒

8ページ目：6.9474873542785645秒
総取得件数：1962
完了：0.4%
残り時間：3時間51分45秒

9ページ目：6.63202166557312秒
総取得件数：2134
完了：0.45%
残り時間：3時間50分21秒

10ページ目：6.250733852386475秒
総取得件数：2384
完了：0.5%
残り時間：3時間47分56秒

11ページ目：5.525048494338989秒
総取得件数：2576
完了：0.55%
残り時間：3時間43分46秒

12ページ目：6.803109407424927秒
総取得件数：2822
完了：0.6%
残り時間：3時間43分48秒

13ページ目：5.779006004333496秒
総取得件数：2955
完了：0.65%
残り時間：3時間41分12秒

14ページ目：5.628916263580322秒
総取得件数：3103
完了：0.7%
残り時間：3時間38分36秒

15ページ目：8.573980808258057秒
総取得件数：3333
完了：0.75%
残り時間：3時間42分50秒

16ページ目：6.161159992218018秒
総取得件数：3524
完了：0.8%
残り時間：3時間41分32秒

17ページ目：6.685344457626343秒
総取得件数

133ページ目：5.908208608627319秒
総取得件数：25383
完了：6.65%
残り時間：3時間29分10秒

134ページ目：6.0093042850494385秒
総取得件数：25528
完了：6.7%
残り時間：3時間28分54秒

135ページ目：7.24438214302063秒
総取得件数：25732
完了：6.75%
残り時間：3時間28分54秒

136ページ目：6.7226481437683105秒
総取得件数：25951
完了：6.8%
残り時間：3時間28分48秒

137ページ目：6.045317888259888秒
総取得件数：26109
完了：6.85%
残り時間：3時間28分32秒

138ページ目：7.07939076423645秒
総取得件数：26304
完了：6.9%
残り時間：3時間28分30秒

139ページ目：6.085777759552002秒
総取得件数：26474
完了：6.95%
残り時間：3時間28分15秒

140ページ目：6.23680567741394秒
総取得件数：26649
完了：7.0%
残り時間：3時間28分2秒

141ページ目：7.037396430969238秒
総取得件数：26876
完了：7.05%
残り時間：3時間27分59秒

142ページ目：6.156262159347534秒
総取得件数：27035
完了：7.1%
残り時間：3時間27分45秒

143ページ目：6.7980732917785645秒
総取得件数：27226
完了：7.15%
残り時間：3時間27分40秒

144ページ目：10.404293537139893秒
総取得件数：27481
完了：7.2%
残り時間：3時間28分21秒

145ページ目：6.5129477977752686秒
総取得件数：27647
完了：7.25%
残り時間：3時間28分11秒

146ページ目：7.037693738937378秒
総取得件数：27820
完了：7.3%
残り時間：3時間28分8秒

147ページ目：5.9005126953125秒
総取得件数：27946
完了：7.35%
残り時間：3時間27分51秒

148ページ目：6.475928068161011秒
総取得件数：28163
完了：7.4%
残り

262ページ目：6.192713975906372秒
総取得件数：47210
完了：13.1%
残り時間：3時間21分8秒

263ページ目：5.8145928382873535秒
総取得件数：47295
完了：13.15%
残り時間：3時間20分53秒

264ページ目：5.99916672706604秒
総取得件数：47406
完了：13.2%
残り時間：3時間20分40秒

265ページ目：6.32887864112854秒
総取得件数：47538
完了：13.25%
残り時間：3時間20分29秒

266ページ目：6.896469593048096秒
総取得件数：47658
完了：13.3%
残り時間：3時間20分22秒

267ページ目：6.986102342605591秒
総取得件数：47827
完了：13.35%
残り時間：3時間20分15秒

268ページ目：7.548402547836304秒
総取得件数：48067
完了：13.4%
残り時間：3時間20分13秒

269ページ目：6.247211694717407秒
総取得件数：48206
完了：13.45%
残り時間：3時間20分1秒

270ページ目：6.241219520568848秒
総取得件数：48311
完了：13.5%
残り時間：3時間19分50秒

271ページ目：6.391942024230957秒
総取得件数：48455
完了：13.55%
残り時間：3時間19分39秒

272ページ目：7.064435720443726秒
総取得件数：48619
完了：13.6%
残り時間：3時間19分33秒

273ページ目：6.683370351791382秒
総取得件数：48764
完了：13.65%
残り時間：3時間19分25秒

274ページ目：6.60511040687561秒
総取得件数：48920
完了：13.7%
残り時間：3時間19分16秒

275ページ目：6.262136220932007秒
総取得件数：49057
完了：13.75%
残り時間：3時間19分5秒

276ページ目：6.726256608963013秒
総取得件数：49212
完了：13.8%
残り時間：3時間18分57秒

277ページ目：6.29389762878418秒
総取得件数：49349


390ページ目：10.87760615348816秒
総取得件数：66206
完了：19.5%
残り時間：3時間10分0秒

391ページ目：6.402939558029175秒
総取得件数：66319
完了：19.55%
残り時間：3時間9分50秒

392ページ目：6.587218523025513秒
総取得件数：66419
完了：19.6%
残り時間：3時間9分41秒

393ページ目：9.885101079940796秒
総取得件数：66530
完了：19.65%
残り時間：3時間9分46秒

394ページ目：7.666301012039185秒
総取得件数：66738
完了：19.7%
残り時間：3時間9分41秒

395ページ目：6.749277114868164秒
総取得件数：66845
完了：19.75%
残り時間：3時間9分32秒

396ページ目：6.945043325424194秒
総取得件数：66987
完了：19.8%
残り時間：3時間9分25秒

397ページ目：7.536223411560059秒
総取得件数：67155
完了：19.85%
残り時間：3時間9分19秒

398ページ目：6.795897960662842秒
総取得件数：67301
完了：19.9%
残り時間：3時間9分11秒

399ページ目：6.469766616821289秒
総取得件数：67420
完了：19.95%
残り時間：3時間9分2秒

400ページ目：6.515577077865601秒
総取得件数：67521
完了：20.0%
残り時間：3時間8分52秒

401ページ目：7.335420608520508秒
総取得件数：67671
完了：20.05%
残り時間：3時間8分46秒

402ページ目：6.7917399406433105秒
総取得件数：67797
完了：20.1%
残り時間：3時間8分38秒

403ページ目：6.735642671585083秒
総取得件数：67983
完了：20.15%
残り時間：3時間8分29秒

404ページ目：7.580893278121948秒
総取得件数：68143
完了：20.2%
残り時間：3時間8分24秒

405ページ目：7.711724758148193秒
総取得件数：68266
完了：20.25%

519ページ目：7.005621433258057秒
総取得件数：81915
完了：25.95%
残り時間：2時間57分26秒

520ページ目：10.876355171203613秒
総取得件数：82020
完了：26.0%
残り時間：2時間57分29秒

521ページ目：6.764648914337158秒
総取得件数：82138
完了：26.05%
残り時間：2時間57分21秒

522ページ目：7.072901248931885秒
総取得件数：82286
完了：26.1%
残り時間：2時間57分14秒

523ページ目：10.44013261795044秒
総取得件数：82387
完了：26.15%
残り時間：2時間57分15秒

524ページ目：8.175336360931396秒
総取得件数：82500
完了：26.2%
残り時間：2時間57分11秒

525ページ目：6.9996771812438965秒
総取得件数：82611
完了：26.25%
残り時間：2時間57分3秒

526ページ目：7.275499582290649秒
総取得件数：82789
完了：26.3%
残り時間：2時間56分56秒

527ページ目：7.704896450042725秒
総取得件数：82908
完了：26.35%
残り時間：2時間56分50秒

528ページ目：6.6516993045806885秒
総取得件数：83013
完了：26.4%
残り時間：2時間56分42秒

529ページ目：6.7458555698394775秒
総取得件数：83131
完了：26.45%
残り時間：2時間56分33秒

530ページ目：6.899217844009399秒
総取得件数：83268
完了：26.5%
残り時間：2時間56分25秒

531ページ目：8.68591570854187秒
総取得件数：83387
完了：26.55%
残り時間：2時間56分22秒

532ページ目：7.439467906951904秒
総取得件数：83553
完了：26.6%
残り時間：2時間56分16秒

533ページ目：8.317774772644043秒
総取得件数：83700
完了：26.65%
残り時間：2時間56分11秒

534ページ目：7.366884231567383秒
総取得

...

1398ページ目：8.452239513397217秒
総取得件数：176816
完了：69.9%
残り時間：1時間21分33秒

1399ページ目：8.384848594665527秒
総取得件数：176923
完了：69.95%
残り時間：1時間21分25秒

1400ページ目：7.847013235092163秒
総取得件数：177024
完了：70.0%
残り時間：1時間21分17秒

1401ページ目：11.417243003845215秒
総取得件数：177123
完了：70.05%
残り時間：1時間21分10秒

1402ページ目：13.881842613220215秒
総取得件数：177213
完了：70.1%
残り時間：1時間21分4秒

1403ページ目：8.264947891235352秒
総取得件数：177298
完了：70.15%
残り時間：1時間20分56秒

1404ページ目：8.599651575088501秒
総取得件数：177414
完了：70.2%
残り時間：1時間20分48秒

1405ページ目：8.117424726486206秒
総取得件数：177511
完了：70.25%
残り時間：1時間20分40秒

1406ページ目：7.992296934127808秒
総取得件数：177614
完了：70.3%
残り時間：1時間20分32秒

1407ページ目：8.821521997451782秒
総取得件数：177711
完了：70.35%
残り時間：1時間20分24秒

1408ページ目：8.326238870620728秒
総取得件数：177804
完了：70.4%
残り時間：1時間20分16秒

1409ページ目：7.968444347381592秒
総取得件数：177894
完了：70.45%
残り時間：1時間20分8秒

1410ページ目：8.603517293930054秒
総取得件数：177989
完了：70.5%
残り時間：1時間20分0秒

1411ページ目：8.70073390007019秒
総取得件数：178104
完了：70.55%
残り時間：1時間19分52秒

1412ページ目：9.148578405380249秒
総取得件数：178216
完了：70.6%
残り時間：1時間19分44秒

1413ペ

1523ページ目：8.170920610427856秒
総取得件数：188264
完了：76.15%
残り時間：1時間5分16秒

1524ページ目：8.26323127746582秒
総取得件数：188337
完了：76.2%
残り時間：1時間5分8秒

1525ページ目：8.52421522140503秒
総取得件数：188404
完了：76.25%
残り時間：1時間5分0秒

1526ページ目：11.485214710235596秒
総取得件数：188470
完了：76.3%
残り時間：1時間4分53秒

1527ページ目：8.37216329574585秒
総取得件数：188534
完了：76.35%
残り時間：1時間4分44秒

1528ページ目：8.917343378067017秒
総取得件数：188592
完了：76.4%
残り時間：1時間4分36秒

1529ページ目：8.437527418136597秒
総取得件数：188674
完了：76.45%
残り時間：1時間4分28秒

1530ページ目：8.294373989105225秒
総取得件数：188753
完了：76.5%
残り時間：1時間4分20秒

1531ページ目：8.275869607925415秒
総取得件数：188817
完了：76.55%
残り時間：1時間4分12秒

1532ページ目：7.935750484466553秒
総取得件数：188883
完了：76.6%
残り時間：1時間4分4秒

1533ページ目：8.612037181854248秒
総取得件数：188946
完了：76.65%
残り時間：1時間3分55秒

1534ページ目：8.866295099258423秒
総取得件数：189039
完了：76.7%
残り時間：1時間3分47秒

1535ページ目：8.19507384300232秒
総取得件数：189126
完了：76.75%
残り時間：1時間3分39秒

1536ページ目：8.41820240020752秒
総取得件数：189198
完了：76.8%
残り時間：1時間3分31秒

1537ページ目：8.304167985916138秒
総取得件数：189270
完了：76.85%
残り時間：1時間3分23秒

1538ページ目：8.9062085151672

1648ページ目：8.998096704483032秒
総取得件数：197737
完了：82.4%
残り時間：0時間48分26秒

1649ページ目：8.295540809631348秒
総取得件数：197804
完了：82.45%
残り時間：0時間48分18秒

1650ページ目：8.124132871627808秒
総取得件数：197873
完了：82.5%
残り時間：0時間48分9秒

1651ページ目：8.696374893188477秒
総取得件数：197951
完了：82.55%
残り時間：0時間48分1秒

1652ページ目：8.648511171340942秒
総取得件数：198019
完了：82.6%
残り時間：0時間47分53秒

1653ページ目：8.646667718887329秒
総取得件数：198084
完了：82.65%
残り時間：0時間47分45秒

1654ページ目：9.054648160934448秒
総取得件数：198143
完了：82.7%
残り時間：0時間47分37秒

1655ページ目：8.108006000518799秒
総取得件数：198216
完了：82.75%
残り時間：0時間47分28秒

1656ページ目：8.551624536514282秒
総取得件数：198280
完了：82.8%
残り時間：0時間47分20秒

1657ページ目：8.522366285324097秒
総取得件数：198358
完了：82.85%
残り時間：0時間47分12秒

1658ページ目：8.3314368724823秒
総取得件数：198440
完了：82.9%
残り時間：0時間47分4秒

1659ページ目：9.056535243988037秒
総取得件数：198506
完了：82.95%
残り時間：0時間46分56秒

1660ページ目：8.237200021743774秒
総取得件数：198575
完了：83.0%
残り時間：0時間46分47秒

1661ページ目：11.445000886917114秒
総取得件数：198641
完了：83.05%
残り時間：0時間46分40秒

1662ページ目：8.341075897216797秒
総取得件数：198714
完了：83.1%
残り時間：0時間46分32秒

1663ページ

1772ページ目：8.450936079025269秒
総取得件数：206825
完了：88.6%
残り時間：0時間31分32秒

1773ページ目：8.359699964523315秒
総取得件数：206919
完了：88.65%
残り時間：0時間31分24秒

1774ページ目：8.537465333938599秒
総取得件数：207011
完了：88.7%
残り時間：0時間31分16秒

1775ページ目：8.379276037216187秒
総取得件数：207072
完了：88.75%
残り時間：0時間31分7秒

1776ページ目：8.69555401802063秒
総取得件数：207168
完了：88.8%
残り時間：0時間30分59秒

1777ページ目：12.49387502670288秒
総取得件数：207236
完了：88.85%
残り時間：0時間30分51秒

1778ページ目：8.293773412704468秒
総取得件数：207296
完了：88.9%
残り時間：0時間30分43秒

1779ページ目：12.259544372558594秒
総取得件数：207363
完了：88.95%
残り時間：0時間30分35秒

1780ページ目：8.188146352767944秒
総取得件数：207434
完了：89.0%
残り時間：0時間30分27秒

1781ページ目：8.119096279144287秒
総取得件数：207500
完了：89.05%
残り時間：0時間30分19秒

1782ページ目：8.929408311843872秒
総取得件数：207565
完了：89.1%
残り時間：0時間30分10秒

1783ページ目：8.200836896896362秒
総取得件数：207632
完了：89.15%
残り時間：0時間30分2秒

1784ページ目：8.398032665252686秒
総取得件数：207715
完了：89.2%
残り時間：0時間29分54秒

1785ページ目：8.230806350708008秒
総取得件数：207781
完了：89.25%
残り時間：0時間29分45秒

1786ページ目：8.627514123916626秒
総取得件数：207849
完了：89.3%
残り時間：0時間29分37秒

1787ペ

1896ページ目：8.214410305023193秒
総取得件数：215643
完了：94.8%
残り時間：0時間14分29秒

1897ページ目：8.960134267807007秒
総取得件数：215710
完了：94.85%
残り時間：0時間14分20秒

1898ページ目：8.362076997756958秒
総取得件数：215782
完了：94.9%
残り時間：0時間14分12秒

1899ページ目：8.45723581314087秒
総取得件数：215848
完了：94.95%
残り時間：0時間14分4秒

1900ページ目：8.283525466918945秒
総取得件数：215915
完了：95.0%
残り時間：0時間13分55秒

1901ページ目：8.256293058395386秒
総取得件数：215982
完了：95.05%
残り時間：0時間13分47秒

1902ページ目：8.570210695266724秒
総取得件数：216077
完了：95.1%
残り時間：0時間13分39秒

1903ページ目：9.87570595741272秒
総取得件数：216146
完了：95.15%
残り時間：0時間13分30秒

1904ページ目：8.45747685432434秒
総取得件数：216213
完了：95.2%
残り時間：0時間13分22秒

1905ページ目：8.175480127334595秒
総取得件数：216280
完了：95.25%
残り時間：0時間13分14秒

1906ページ目：8.612764596939087秒
総取得件数：216356
完了：95.3%
残り時間：0時間13分5秒

1907ページ目：8.312445878982544秒
総取得件数：216419
完了：95.35%
残り時間：0時間12分57秒

1908ページ目：8.69132685661316秒
総取得件数：216501
完了：95.4%
残り時間：0時間12分48秒

1909ページ目：12.841724395751953秒
総取得件数：216576
完了：95.45%
残り時間：0時間12分40秒

1910ページ目：8.312045335769653秒
総取得件数：216652
完了：95.5%
残り時間：0時間12分32秒

1911ページ目

In [119]:
columns = ['カテゴリ','建物名','住所','最寄り駅1','最寄り駅2','最寄り駅3','築年数','階数','階','賃料','管理費','敷金','礼金','間取り','専有面積','url']
suumo_tokyo = pd.DataFrame(data_samples, columns=columns)

In [120]:
suumo_tokyo

,カテゴリ,建物名,住所,最寄り駅1,最寄り駅2,最寄り駅3,築年数,階数,階,賃料,管理費,敷金,礼金,間取り,専有面積,url
0,賃貸アパート,プチ・ボヌール,東京都大田区南蒲田３,京急空港線/糀谷駅 歩7分,京急本線/京急蒲田駅 歩10分,京急本線/雑色駅 歩16分,築4年,2階建,2階,14.8万円,5000円,14.8万円,14.8万円,42DK,48.87m2,https://suumo.jp/chintai/jnc_000046696863/?bc=...
1,賃貸一戸建て,荒川戸建,東京都荒川区荒川２,東京メトロ千代田線/町屋駅 歩5分,京成本線/新三河島駅 歩12分,ＪＲ常磐線/三河島駅 歩15分,築10年,3階建,1-3階,19.5万円,-,39万円,19.5万円,3SLDK,97.95m2,https://suumo.jp/chintai/jnc_000076009318/?bc=...
2,賃貸一戸建て,荒川戸建,東京都荒川区荒川２,東京メトロ千代田線/町屋駅 歩5分,京成本線/新三河島駅 歩12分,ＪＲ常磐線/三河島駅 歩15分,築10年,3階建,1-3階,19.5万円,-,39万円,19.5万円,3SLDK,97.95m2,https://suumo.jp/chintai/jnc_000076047600/?bc=...
3,賃貸マンション,ティモーネ上野の森ノース,東京都荒川区東日暮里４,ＪＲ山手線/鶯谷駅 歩6分,東京メトロ日比谷線/入谷駅 歩11分,ＪＲ山手線/日暮里駅 歩13分,築5年,11階建,2階,15.2万円,10000円,15.2万円,15.2万円,2LDK,52.04m2,https://suumo.jp/chintai/jnc_000077318759/?bc=...
4,賃貸アパート,東京メトロ日比谷線 三ノ輪駅 3階建 築13年,東京都荒川区南千住５,東京メトロ日比谷線/三ノ輪駅 歩5分,ＪＲ常磐線/南千住駅 歩7分,京成本線/千住大橋駅 歩15分,築13年,3階建,3階,11.3万円,2000円,11.3万円,11.3万円,1LDK,43.81m2,https://suumo.jp/chintai/jnc_000077330023/?bc=...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222918,賃貸一戸建て,中澤貸家,東京都江戸川区江戸川２,都営新宿線/瑞江駅 歩14分,,,築51年,2階建,1-2階,8.4万円,-,16.8万円,-,3K,52.87m2,https://suumo.jp/chintai/jnc_000070035188/?bc=...
222919,賃貸マンション,東京メトロ副都心線 雑司が谷駅 7階建 築37年,東京都豊島区高田１,東京メトロ副都心線/雑司が谷駅 歩4分,ＪＲ山手線/目白駅 歩11分,東京メトロ有楽町線/護国寺駅 歩15分,築37年,7階建,7階,9.2万円,8000円,9.2万円,9.2万円,1DK,26.04m2,https://suumo.jp/chintai/jnc_000074167859/?bc=...
222920,賃貸アパート,ハイツ南大井,東京都品川区南大井５,京急本線/立会川駅 歩7分,ＪＲ京浜東北線/大井町駅 歩14分,ＪＲ京浜東北線/大森駅 歩14分,築20年,2階建,1階,8.5万円,3000円,8.5万円,8.5万円,ワンルーム,28.56m2,https://suumo.jp/chintai/jnc_000043176296/?bc=...
222921,賃貸マンション,芦間ハイツ,東京都葛飾区東新小岩１,ＪＲ総武線/新小岩駅 歩3分,,,築45年,3階建,3階,7.5万円,-,7.5万円,7.5万円,1DK,30.36m2,https://suumo.jp/chintai/jnc_000040819257/?bc=...


In [3]:
suumo_tokyo.to_csv(f'suumo{max_page}.csv', columns=columns, index=False)

NameError: name 'suumo_tokyo' is not defined

In [6]:
max_page = 2000
suumo_tokyo = pd.read_csv(f'suumo{max_page}.csv')

In [7]:
# 管理費（正規表現）
suumo_tokyo['管理費']

# 管理費の'-'を0に置き換える
suumo_tokyo.loc[suumo_tokyo['管理費']=="-",'管理費'] = suumo_tokyo.loc[suumo_tokyo['管理費']=="-",'管理費'].replace('-','0')

# 数字+文字→数字
def numobj_num(x):
    return re.sub(r'([0-9]+)\w+',r'\1',x)

# ????円→????
suumo_tokyo['管理費'] = suumo_tokyo['管理費'].map(lambda x: numobj_num(x))
# 整数型に変更
suumo_tokyo['管理費'] = suumo_tokyo['管理費'].astype(int)

In [8]:
# 築年数（正規表現）

# 新築は築0年ということにする
suumo_tokyo.loc[suumo_tokyo['築年数']=='新築','築年数'] = '築0年'

# 文字+数字+文字→数字
def objnumobj_num(x):
    return re.sub(r'\w+?([0-9]+)\w+',r'\1',x)

# 築??年(以上)→??
# 99階以上は99階に圧縮されます
suumo_tokyo['築年数'] = suumo_tokyo['築年数'] = suumo_tokyo['築年数'].map(lambda x: objnumobj_num(x))

# 
suumo_tokyo['築年数'] = suumo_tokyo['築年数'].astype(int)

In [9]:
# 地下変数（正規表現）

# 平屋は1階建ということにする
suumo_tokyo.loc[suumo_tokyo['階数']=='平屋','階数'] = '1階建'

# この文字列ありますか？関数
def search_object(search,object):
    return bool(re.search(search,object))

# 地下何階までありますか？関数
def underground(x):
    return search_object('地下',x)*objnumobj_num(x)+(1-search_object('地下',x))*'0'
    
# 地下がなければ０、あるなら何階まであるか
suumo_tokyo['地下'] = suumo_tokyo['階数'].map(lambda x: underground(x))

# int型に変換
suumo_tokyo['地下'] = suumo_tokyo['地下'].astype('int')

In [10]:
# 地上変数(正規表現)
suumo_tokyo['階数']

# 平屋は1階建ということにする
suumo_tokyo.loc[suumo_tokyo['階数']=='平屋','階数'] = '1階建'

# 最後の数字列を取り出す
def lastnum(x):
    return re.findall('[0-9]+',x)[-1]

# ??階建→??
suumo_tokyo['地上'] = suumo_tokyo['階数'].map(lambda x: lastnum(x))

# int型に変換
suumo_tokyo['地上'] = suumo_tokyo['地上'].astype(int)

In [11]:
# 階数変数は地下地上の合算にしておく
suumo_tokyo['階数'] = suumo_tokyo['地上'] + suumo_tokyo['地下']

In [12]:
# 階変数(正規表現)

# - は1階ってことにする
suumo_tokyo.loc[suumo_tokyo['階']=='-','階'] = '1階'

# (文字)数字を取り出す
def get_objnum(x):
    return re.search(r'([A-Z]?)([0-9]+)',x).group()

# 階の前処理 -があればその後ろ、なければ(文字)数字
def floor(x):
    return search_object('-',x)*lastnum(x)+(1-search_object('-',x))*get_objnum(x)
# B? → -?,Bがなければそのまま
def basement_floor(x):
    return search_object('B',x)*('-'+re.search(r'[0-9]+',x).group())+(1-search_object('B',x))*(x)
# M? → ?.5 Mがなければそのまま
def middle_floor(x):
    return search_object('M',x)*(re.search(r'[0-9]+',x).group()+'.5')+(1-search_object('M',x))*(x)
   
# 上の処理を適用する
suumo_tokyo['階'] = suumo_tokyo['階'].map(lambda x: floor(x)).map(lambda x: basement_floor(x)).map(lambda x: middle_floor(x))

# float型に変換
suumo_tokyo['階'] = suumo_tokyo['階'].astype(float)

In [13]:
# 賃料変数(正規表現)
# 10000倍するかは諸説

# ???万円→???
def drop_man(x):
    return re.sub(r'([0-9]+)万円',r'\1',x)

# 賃料から万円を除去
suumo_tokyo['賃料'] = suumo_tokyo['賃料'].map(lambda x: drop_man(x))
# float型に変換
suumo_tokyo['賃料'] = suumo_tokyo['賃料'].astype(float)

In [14]:
# 礼金変数(正規表現)
# 10000倍するかは諸説

# '-'は0万円ってことにする
suumo_tokyo.loc[suumo_tokyo['敷金']=='-','敷金'] = '0万円'

# 敷金から万円を除去
suumo_tokyo['敷金'] = suumo_tokyo['敷金'].map(lambda x: drop_man(x))
# float型に変換
suumo_tokyo['敷金'] = suumo_tokyo['敷金'].astype(float)

In [15]:
# 礼金変数(正規表現)
# 10000倍するかは諸説

# '-'は0万円ってことにする
suumo_tokyo.loc[suumo_tokyo['礼金']=='-','礼金'] = '0万円'

# 礼金から万円を除去
suumo_tokyo['礼金'] = suumo_tokyo['礼金'].map(lambda x: drop_man(x))
# float型に変換
suumo_tokyo['礼金'] = suumo_tokyo['礼金'].astype(float)

In [16]:
# 間取り変換案
# そのまま
# 部屋数、onehot(L,D,K)に変換

In [17]:
# 専有面積変数（正規表現）

# 英字以降を切り落とす
def drop_behind_alfa(x):
    return re.sub('[a-z]\w+','',x)
# ????m2 → ????
suumo_tokyo['専有面積'] = suumo_tokyo['専有面積'].map(lambda x: drop_behind_alfa(x))
# float型に変換
suumo_tokyo['専有面積'] = suumo_tokyo['専有面積'].astype(float)

In [18]:
# 住所変数（正規表現）

# ～都,～区,～ に分けたい
# ついでに[都道府県][市区町村][以下]に対応したい

x = suumo_tokyo.loc[0,'住所']

# 住所を(都道府県)(市区町村)(市区町村以下)に分けて取り出す
def split_address(x):
    a,b,c = re.search('(...??[都道府県])(.+?[市区町村])(.+)',x).groups()
    return a,b,c

# 一応分割できたっぽい
suumo_tokyo[['都道府県','市区町村','市区町村以下']] = suumo_tokyo.apply(lambda x:split_address(x.住所),axis=1,result_type='expand')

In [19]:
# 最寄り駅変数（正規表現）

# 最寄り駅から路線と駅を取り出す
suumo_tokyo[['路線','駅']] = suumo_tokyo.apply(lambda x:re.search('(.+)/(.+?)\s(.+)',x.最寄り駅1).groups()[0:2],axis=1,result_type='expand')

def how_to_station():
    # 移動手段は3パターン
    how_to_station = ['歩','バス','車']
    for by in how_to_station:
        # 移動手段変数を生成
        suumo_tokyo[f'{by}'] = np.zeros(suumo_tokyo.shape[0])
        # その移動手段があるなら移動手段変数に格納する
        suumo_tokyo.loc[suumo_tokyo['最寄り駅1'].map(lambda x: search_object(f'{by}[0-9]+分',x)),f'{by}'] = suumo_tokyo.loc[suumo_tokyo['最寄り駅1'].map(lambda x: search_object(f'{by}[0-9]+分',x)),'最寄り駅1'].map(lambda x: re.search(f'{by}([0-9]+)分',x).group(1))
        # 所要時間をint型に変換
        suumo_tokyo[f'{by}'] = suumo_tokyo[f'{by}'].astype(int)

# 移動手段ごとに所用時間変数を作る
how_to_station()

In [20]:
suumo_tokyo

,カテゴリ,建物名,住所,最寄り駅1,最寄り駅2,最寄り駅3,築年数,階数,階,賃料,...,地下,地上,都道府県,市区町村,市区町村以下,路線,駅,歩,バス,車
0,賃貸アパート,プチ・ボヌール,東京都大田区南蒲田３,京急空港線/糀谷駅 歩7分,京急本線/京急蒲田駅 歩10分,京急本線/雑色駅 歩16分,4,2,2.0,14.8,...,0,2,東京都,大田区,南蒲田３,京急空港線,糀谷駅,7,0,0
1,賃貸一戸建て,荒川戸建,東京都荒川区荒川２,東京メトロ千代田線/町屋駅 歩5分,京成本線/新三河島駅 歩12分,ＪＲ常磐線/三河島駅 歩15分,10,3,3.0,19.5,...,0,3,東京都,荒川区,荒川２,東京メトロ千代田線,町屋駅,5,0,0
2,賃貸一戸建て,荒川戸建,東京都荒川区荒川２,東京メトロ千代田線/町屋駅 歩5分,京成本線/新三河島駅 歩12分,ＪＲ常磐線/三河島駅 歩15分,10,3,3.0,19.5,...,0,3,東京都,荒川区,荒川２,東京メトロ千代田線,町屋駅,5,0,0
3,賃貸マンション,ティモーネ上野の森ノース,東京都荒川区東日暮里４,ＪＲ山手線/鶯谷駅 歩6分,東京メトロ日比谷線/入谷駅 歩11分,ＪＲ山手線/日暮里駅 歩13分,5,11,2.0,15.2,...,0,11,東京都,荒川区,東日暮里４,ＪＲ山手線,鶯谷駅,6,0,0
4,賃貸アパート,東京メトロ日比谷線 三ノ輪駅 3階建 築13年,東京都荒川区南千住５,東京メトロ日比谷線/三ノ輪駅 歩5分,ＪＲ常磐線/南千住駅 歩7分,京成本線/千住大橋駅 歩15分,13,3,3.0,11.3,...,0,3,東京都,荒川区,南千住５,東京メトロ日比谷線,三ノ輪駅,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222918,賃貸一戸建て,中澤貸家,東京都江戸川区江戸川２,都営新宿線/瑞江駅 歩14分,NaN,NaN,51,2,2.0,8.4,...,0,2,東京都,江戸川区,江戸川２,都営新宿線,瑞江駅,14,0,0
222919,賃貸マンション,東京メトロ副都心線 雑司が谷駅 7階建 築37年,東京都豊島区高田１,東京メトロ副都心線/雑司が谷駅 歩4分,ＪＲ山手線/目白駅 歩11分,東京メトロ有楽町線/護国寺駅 歩15分,37,7,7.0,9.2,...,0,7,東京都,豊島区,高田１,東京メトロ副都心線,雑司が谷駅,4,0,0
222920,賃貸アパート,ハイツ南大井,東京都品川区南大井５,京急本線/立会川駅 歩7分,ＪＲ京浜東北線/大井町駅 歩14分,ＪＲ京浜東北線/大森駅 歩14分,20,2,1.0,8.5,...,0,2,東京都,品川区,南大井５,京急本線,立会川駅,7,0,0
222921,賃貸マンション,芦間ハイツ,東京都葛飾区東新小岩１,ＪＲ総武線/新小岩駅 歩3分,NaN,NaN,45,3,3.0,7.5,...,0,3,東京都,葛飾区,東新小岩１,ＪＲ総武線,新小岩駅,3,0,0


In [22]:
# 順番を見やすいように入れ替える
columns = ['カテゴリ','建物名','都道府県','市区町村','市区町村以下','路線','駅','歩','バス','車','築年数','地上','地下','階数','階','賃料','管理費','敷金','礼金','間取り','専有面積','url']
suumo_tokyo_shaped = suumo_tokyo[columns]
# 重複を削除する（重複10000くらいありましたね…）
suumo_tokyo_shaped = suumo_tokyo_shaped[suumo_tokyo_shaped.duplicated() == False]
# 保存
suumo_tokyo_shaped.to_csv('SuumoTokyoShaped20221012.csv', index=False)

In [23]:
columns = ['カテゴリ','建物名','都道府県','市区町村','市区町村以下','路線','駅','歩','バス','車','築年数','地上','地下','階数','階','賃料','管理費','敷金','礼金','間取り','専有面積']
suumo_tokyo_shaped[columns]

# suumo_tokyo_shaped['階']/suumo_tokyo_shaped['階数']

,カテゴリ,建物名,都道府県,市区町村,市区町村以下,路線,駅,歩,バス,車,...,地上,地下,階数,階,賃料,管理費,敷金,礼金,間取り,専有面積
0,賃貸アパート,プチ・ボヌール,東京都,大田区,南蒲田３,京急空港線,糀谷駅,7,0,0,...,2,0,2,2.0,14.8,5000,14.8,14.8,42DK,48.87
1,賃貸一戸建て,荒川戸建,東京都,荒川区,荒川２,東京メトロ千代田線,町屋駅,5,0,0,...,3,0,3,3.0,19.5,0,39.0,19.5,3SLDK,97.95
2,賃貸一戸建て,荒川戸建,東京都,荒川区,荒川２,東京メトロ千代田線,町屋駅,5,0,0,...,3,0,3,3.0,19.5,0,39.0,19.5,3SLDK,97.95
3,賃貸マンション,ティモーネ上野の森ノース,東京都,荒川区,東日暮里４,ＪＲ山手線,鶯谷駅,6,0,0,...,11,0,11,2.0,15.2,10000,15.2,15.2,2LDK,52.04
4,賃貸アパート,東京メトロ日比谷線 三ノ輪駅 3階建 築13年,東京都,荒川区,南千住５,東京メトロ日比谷線,三ノ輪駅,5,0,0,...,3,0,3,3.0,11.3,2000,11.3,11.3,1LDK,43.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222913,賃貸マンション,メゾンスギモト,東京都,江戸川区,東小岩１,ＪＲ総武線,小岩駅,20,0,0,...,3,0,3,3.0,8.5,0,8.5,8.5,2DK,40.00
222915,賃貸マンション,大場ビル,東京都,江戸川区,西一之江２,都営新宿線,一之江駅,26,0,0,...,3,0,3,2.0,8.6,4000,0.0,0.0,2LDK,47.25
222918,賃貸一戸建て,中澤貸家,東京都,江戸川区,江戸川２,都営新宿線,瑞江駅,14,0,0,...,2,0,2,2.0,8.4,0,16.8,0.0,3K,52.87
222919,賃貸マンション,東京メトロ副都心線 雑司が谷駅 7階建 築37年,東京都,豊島区,高田１,東京メトロ副都心線,雑司が谷駅,4,0,0,...,7,0,7,7.0,9.2,8000,9.2,9.2,1DK,26.04


In [24]:
suumo_tokyo_shaped

,カテゴリ,建物名,都道府県,市区町村,市区町村以下,路線,駅,歩,バス,車,...,地下,階数,階,賃料,管理費,敷金,礼金,間取り,専有面積,url
0,賃貸アパート,プチ・ボヌール,東京都,大田区,南蒲田３,京急空港線,糀谷駅,7,0,0,...,0,2,2.0,14.8,5000,14.8,14.8,42DK,48.87,https://suumo.jp/chintai/jnc_000046696863/?bc=...
1,賃貸一戸建て,荒川戸建,東京都,荒川区,荒川２,東京メトロ千代田線,町屋駅,5,0,0,...,0,3,3.0,19.5,0,39.0,19.5,3SLDK,97.95,https://suumo.jp/chintai/jnc_000076009318/?bc=...
2,賃貸一戸建て,荒川戸建,東京都,荒川区,荒川２,東京メトロ千代田線,町屋駅,5,0,0,...,0,3,3.0,19.5,0,39.0,19.5,3SLDK,97.95,https://suumo.jp/chintai/jnc_000076047600/?bc=...
3,賃貸マンション,ティモーネ上野の森ノース,東京都,荒川区,東日暮里４,ＪＲ山手線,鶯谷駅,6,0,0,...,0,11,2.0,15.2,10000,15.2,15.2,2LDK,52.04,https://suumo.jp/chintai/jnc_000077318759/?bc=...
4,賃貸アパート,東京メトロ日比谷線 三ノ輪駅 3階建 築13年,東京都,荒川区,南千住５,東京メトロ日比谷線,三ノ輪駅,5,0,0,...,0,3,3.0,11.3,2000,11.3,11.3,1LDK,43.81,https://suumo.jp/chintai/jnc_000077330023/?bc=...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222913,賃貸マンション,メゾンスギモト,東京都,江戸川区,東小岩１,ＪＲ総武線,小岩駅,20,0,0,...,0,3,3.0,8.5,0,8.5,8.5,2DK,40.00,https://suumo.jp/chintai/jnc_000069889741/?bc=...
222915,賃貸マンション,大場ビル,東京都,江戸川区,西一之江２,都営新宿線,一之江駅,26,0,0,...,0,3,2.0,8.6,4000,0.0,0.0,2LDK,47.25,https://suumo.jp/chintai/jnc_000019146531/?bc=...
222918,賃貸一戸建て,中澤貸家,東京都,江戸川区,江戸川２,都営新宿線,瑞江駅,14,0,0,...,0,2,2.0,8.4,0,16.8,0.0,3K,52.87,https://suumo.jp/chintai/jnc_000070035188/?bc=...
222919,賃貸マンション,東京メトロ副都心線 雑司が谷駅 7階建 築37年,東京都,豊島区,高田１,東京メトロ副都心線,雑司が谷駅,4,0,0,...,0,7,7.0,9.2,8000,9.2,9.2,1DK,26.04,https://suumo.jp/chintai/jnc_000074167859/?bc=...
